In [2]:
import pandas as pd
import numpy as np
import cython
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import make_classification

data = make_classification(n_samples=1000, n_features=20, n_informative=16, n_redundant=4, random_state=42)
X, y = data
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y = pd.Series(y, name='target')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from CalculateMetrics import calculate_all_metrics
from EnderClassifier import EnderClassifier

print("ENDER CLASSIFIER NORMAL")
ender = EnderClassifier(verbose=True, n_rules=10, use_gradient=False)
ender.fit(X_train, y_train)

ENDER CLASSIFIER NORMAL
Default rule: [0.0, -0.0]
####################################################################################
Rule: 1
	feature_2 in [0.537934390169948, 99999999999999999]
	feature_14 in [-99999999999999999, 1.777028192334217]
	feature_16 in [-99999999999999999, 1.6679856004407545]
	feature_10 in [-99999999999999999, 3.2491830324383644]
	feature_0 in [-99999999999999999, 4.705576021045976]
	feature_7 in [-10.887088759569243, 99999999999999999]
	feature_18 in [-99999999999999999, 6.255369842528556]
	feature_1 in [-4.536887114417905, 99999999999999999]
	feature_3 in [-5.0334443573758785, 99999999999999999]
	feature_4 in [-99999999999999999, 3.871499133304778]
	feature_13 in [-3.388930644543708, 99999999999999999]
=> vote for class 0 with weight 0.8252129146403612
[0.8252129146403612, -0.8252129146403612]

####################################################################################
Rule: 2
	feature_14 in [3.797967467888083, 99999999999999999]
	feature_3 in 

In [4]:
y_train_pred = ender.predict(X_train)
y_test_pred = ender.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Normal Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Normal Ender Classifier
Accuracy: 0.82

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84        99
           1       0.89      0.74      0.81       101

    accuracy                           0.82       200
   macro avg       0.83      0.83      0.82       200
weighted avg       0.83      0.82      0.82       200

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    90                  9                     # True Negative (TN), False Positive (FP)
Endometriosis       26                  75                    # False Negative (FN), True Positive (TP)


In [5]:
from EnderClassifierBounded import EnderClassifier

print("ENDER CLASSIFIER BOUNDED")
ender = EnderClassifier(verbose=True, n_rules=10, use_gradient=False)
ender.fit(X_train, y_train)

ENDER CLASSIFIER BOUNDED
Initializing for rule...
Default rule: [0.0, -0.0]
####################################################################################
Rule: 1
Initializing for rule...
Lower bound 66.87969469879319 is less than empirical risk inf for rule with 1 conditions. Continuing search.
Lower bound 151.1304183783332 is less than empirical risk inf for rule with 2 conditions. Continuing search.
Lower bound 184.3801780345259 is less than empirical risk inf for rule with 3 conditions. Continuing search.
Lower bound 192.67069334973075 is less than empirical risk inf for rule with 4 conditions. Continuing search.
Lower bound 193.1712589179383 is less than empirical risk inf for rule with 5 conditions. Continuing search.
Lower bound 193.1712589179383 is less than empirical risk inf for rule with 6 conditions. Continuing search.
Lower bound 193.1712589179383 is less than empirical risk inf for rule with 7 conditions. Continuing search.
Lower bound 193.1712589179383 is less than

In [6]:
y_train_pred = ender.predict(X_train)
y_test_pred = ender.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Bounded Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Bounded Ender Classifier
Accuracy: 0.81

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.78      0.85      0.81        99
           1       0.84      0.76      0.80       101

    accuracy                           0.81       200
   macro avg       0.81      0.81      0.80       200
weighted avg       0.81      0.81      0.80       200

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    84                  15                    # True Negative (TN), False Positive (FP)
Endometriosis       24                  77                    # False Negative (FN), True Positive (TP)


In [1]:
from CalculateMetrics import calculate_all_metrics
from EnderClassifierModified import EnderClassifier

print("EnderClassifierModified")
enderM = EnderClassifier(verbose=False, n_rules=20, use_gradient=False)
enderM.fit(X_train, y_train)

EnderClassifierModified


NameError: name 'X_train' is not defined

In [ ]:
y_train_pred = enderM.predict(X_train)
y_test_pred = enderM.predict(X_test)

train_metrics = calculate_all_metrics(y_train, y_train_pred)
test_metrics = calculate_all_metrics(y_test, y_test_pred)

y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

print("Modified Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Modified Ender Classifier
Accuracy: 0.81

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       150
           1       0.64      0.52      0.57        48

    accuracy                           0.81       198
   macro avg       0.75      0.71      0.73       198
weighted avg       0.80      0.81      0.81       198

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    136                 14                    # True Negative (TN), False Positive (FP)
Endometriosis       23                  25                    # False Negative (FN), True Positive (TP)


In [ ]:
for n in range(15, 45):
    ender = EnderClassifier(verbose=False, n_rules=n, use_gradient=False)
    ender.fit(X_train, y_train)
    y_train_pred = ender.predict(X_train)
    y_train_pred = np.array([np.argmax(pred) for pred in y_train_pred])
    y_test_pred = ender.predict(X_test)
    y_pred = np.array([np.argmax(pred) for pred in y_test_pred])
    print(f"EnderClassifier with {n} rules")
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred):.2f}")
    print("================================")

EnderClassifier with 15 rules
Test Accuracy: 0.82
Train Accuracy: 0.94
EnderClassifier with 16 rules
Test Accuracy: 0.82
Train Accuracy: 0.94
EnderClassifier with 17 rules
Test Accuracy: 0.83
Train Accuracy: 0.95
EnderClassifier with 18 rules
Test Accuracy: 0.83
Train Accuracy: 0.95
EnderClassifier with 19 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 20 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 21 rules
Test Accuracy: 0.83
Train Accuracy: 0.96
EnderClassifier with 22 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 23 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 24 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 25 rules
Test Accuracy: 0.82
Train Accuracy: 0.97
EnderClassifier with 26 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 27 rules
Test Accuracy: 0.83
Train Accuracy: 0.97
EnderClassifier with 28 rules
Test Accuracy: 0.83
Train Accuracy: 0.98
EnderC

In [ ]:
from time import time
ender = EnderClassifier(verbose=False, n_rules=41, use_gradient=False)
start_time = time()
ender.fit(X_train, y_train)
# y_train_pred = ender.predict(X_train)
# y_train_pred = np.array([np.argmax(pred) for pred in y_train_pred])
y_test_pred = ender.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in y_test_pred])
print(f"Training time for 14 rules: {time() - start_time:.2f} seconds")

print("Modified Ender Classifier")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
print("Classification Report - Test:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
labels = ['No Endometriosis', 'Endometriosis']
print("Confusion Matrix:")
print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

Training time for 14 rules: 0.51 seconds
Modified Ender Classifier
Accuracy: 0.84

Classification Report - Test:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       151
           1       0.75      0.50      0.60        48

    accuracy                           0.84       199
   macro avg       0.80      0.72      0.75       199
weighted avg       0.83      0.84      0.83       199

Confusion Matrix:
                    No Endometriosis    Endometriosis       
No Endometriosis    143                 8                     # True Negative (TN), False Positive (FP)
Endometriosis       24                  24                    # False Negative (FN), True Positive (TP)


In [ ]:
# from EnderRegressor import EnderRegressor
# from CalculateMetrics import calculate_all_metrics

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# df = pd.read_csv("/home/maciej/Desktop/Maciej/endometriosis/endometriosis.csv")
# X = df.drop(columns=['target'])
# y = df['target'].astype('int')
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# ender = EnderRegressor(verbose=False)
# ender.fit(X_train, y_train)

In [ ]:
# ender.evaluate_all_rules()
# ender.rules

In [ ]:
# y_train_pred = ender.predict(X_train)
# y_test_pred = ender.predict(X_test)

# train_metrics = calculate_all_metrics(y_train, y_train_pred)
# test_metrics = calculate_all_metrics(y_test, y_test_pred)

In [ ]:
# y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

# print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
# print("Classification Report - Test:")
# print(classification_report(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# labels = ['No Endometriosis', 'Endometriosis']
# print("Confusion Matrix:")
# print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
# print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
# print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")

In [ ]:
# from EnderClassifier import EnderClassifier as ECOG

In [ ]:
# ender2 = ECOG(verbose=False)
# ender2.fit(X_train, y_train)

In [ ]:
# ender2.evaluate_all_rules()

In [ ]:
# ender = ender2

In [ ]:
# y_train_pred = ender.predict(X_train)
# y_test_pred = ender.predict(X_test)
# train_metrics = calculate_all_metrics(y_train, y_train_pred)
# test_metrics = calculate_all_metrics(y_test, y_test_pred)

In [ ]:
# y_pred = np.array([np.argmax(pred) for pred in y_test_pred])

# print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")
# print("Classification Report - Test:")
# print(classification_report(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# labels = ['No Endometriosis', 'Endometriosis']
# print("Confusion Matrix:")
# print(f"{'':<20}{labels[0]:<20}{labels[1]:<20}")
# print(f"{labels[0]:<20}{cm[0, 0]:<20}{cm[0, 1]:<20}  # True Negative (TN), False Positive (FP)")
# print(f"{labels[1]:<20}{cm[1, 0]:<20}{cm[1, 1]:<20}  # False Negative (FN), True Positive (TP)")